In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset, random_split

# Data Loading and Preprocessing

---

In [2]:
import json
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Load data function
def load_data(file_path):
    with open(file_path, 'r') as f:
        return [json.loads(line) for line in f]

domain1_data = load_data('domain1_train.json')
domain2_data = load_data('domain2_train.json')
domain2_data = [entry for entry in domain2_data if len(entry['text']) > 0]


In [3]:
# Preprocess data function
def preprocess_data(data, domain_label):
    for sample in data:
        sample['domain'] = domain_label
    return data

domain1_data = preprocess_data(domain1_data, 0)  # Label for domain1
domain2_data = preprocess_data(domain2_data, 1)  # Label for domain2


In [4]:
# Split the data
train_data_domain, valid_data_domain = train_test_split(domain1_data+domain2_data, test_size=0.2, random_state=42)

train_data = train_data_domain
valid_data = valid_data_domain

# Create PyTorch Datasets and DataLoaders

In [18]:
# Define dataset
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = torch.tensor(self.data[idx]['text'])
        label = torch.tensor(self.data[idx]['label'])
        domain = torch.tensor(self.data[idx]['domain'])
        return text, label, domain


# Custom collate function
def collate_batch(batch):
    texts, labels, domains = zip(*batch)
    text_lengths = [len(txt) for txt in texts]
    texts = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels).float()  # Convert labels to float
    domains = torch.tensor(domains).float()  # Convert domains to float for BCEWithLogitsLoss
    return texts, labels, domains, text_lengths

# DataLoader with the custom collate function
train_dataset = TextDataset(train_data)
valid_dataset = TextDataset(valid_data)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# Model Definition---Domain Adaption

In [6]:



class GradientReversalFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

def reverse_gradient(x, alpha=1.2):
    return GradientReversalFunction.apply(x, alpha)



In [8]:
class BiLSTMClassifierWithDA(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional=True, dropout=0.5):
        super(BiLSTMClassifierWithDA, self).__init__()

        # Text embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # BiLSTM layer
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)

        # Classifier layer
        self.fc = nn.Linear(hidden_dim*2, output_dim)  # x2 for bidirectional

        # Domain Discriminator layer with added capacity
        self.domain_discriminator = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def extract_features(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        return hidden

    def classify(self, features):
        return self.fc(self.dropout(features)).squeeze(1)

    def domain_discriminator_forward(self, reversed_features):
        return self.domain_discriminator(reversed_features).squeeze(1)

    def forward(self, text, text_lengths, apply_discriminator=False, alpha=1.2):
        features = self.extract_features(text, text_lengths)

        if apply_discriminator:
            reversed_features = reverse_gradient(features, alpha=alpha)  # Apply gradient reversal
            return self.domain_discriminator_forward(reversed_features)
        else:
            return self.classify(features)


# Loss Function and Optimizer

In [9]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Extract model counts for domain 2
models_domain2 = [entry['label'] for entry in domain2_data]
class_counts_domain2 = [models_domain2.count(float(i)) for i in range(int(max(models_domain2)) + 1)]

# Compute class weights for domain 2
total_samples_domain2 = sum(class_counts_domain2)
class_weights_domain2 = torch.tensor([total_samples_domain2 / count for count in class_counts_domain2]).float().to(device)

def get_batch_weights(labels, class_weights_domain2):
    return class_weights_domain2[labels.long()].unsqueeze(1)

In [12]:

# Instantiate the model
model = BiLSTMClassifierWithDA(vocab_size=5000, embedding_dim=128, hidden_dim=256, output_dim=1, n_layers=2).to(device)

# Classification criterion
classification_criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_domain2) ##change

# Domain adaptation criterion
domain_adaptation_criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = optim.Adam(model.parameters())

# Binary accuracy function
def binary_accuracy(predictions, y):
    rounded_preds = torch.round(torch.sigmoid(predictions)).squeeze()  # Ensure it's a 1D tensor
    correct = (rounded_preds == y).float()
    return correct.sum() / len(correct)


from sklearn.metrics import f1_score

def compute_f1(predictions, labels):
    # Convert predictions to binary
    preds_binary = torch.round(torch.sigmoid(predictions))
    preds_binary = preds_binary.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()

    return f1_score(labels, preds_binary)



# Training Loop

In [13]:
from tqdm import tqdm
num_epochs = 23
alpha = 7  # Gradient reversal scale. Adjust based on your needs.


clip_value = 1

for epoch in range(num_epochs):
    epoch_loss_classifier = 0
    epoch_acc_classifier = 0
    epoch_f1_classifier = 0
    epoch_loss_domain = 0
    epoch_acc_domain = 0
    epoch_f1_domain = 0

    model.train()

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        texts, labels, domains, text_lengths = batch
        texts = texts.to(device)
        labels = labels.to(device)
        domains = domains.to(device)
        text_lengths = torch.tensor(text_lengths).long()  # Keep it on CPU

        batch_weights = get_batch_weights(labels, class_weights_domain2).to(device)

        optimizer.zero_grad()

        # Get the classifier's predictions
        predictions = model(texts, text_lengths)
        batch_criterion = nn.BCEWithLogitsLoss(pos_weight=batch_weights.squeeze()).to(device)
        loss_classifier = batch_criterion(predictions, labels)
        acc_classifier = binary_accuracy(predictions, labels)
        f1_classifier = compute_f1(predictions, labels)

        # Get the domain discriminator's predictions
        domain_predictions = model(texts, text_lengths, apply_discriminator=True, alpha=alpha)
        loss_domain = domain_adaptation_criterion(domain_predictions, domains)
        acc_domain = binary_accuracy(domain_predictions, domains)
        f1_domain = compute_f1(domain_predictions, domains)

        combined_loss = loss_classifier + loss_domain  # Removed reverse_gradient as we should apply gradient reversal on features and not the loss
        combined_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        epoch_loss_classifier += loss_classifier.item()
        epoch_acc_classifier += acc_classifier.item()
        epoch_f1_classifier += f1_classifier.item()
        epoch_loss_domain += loss_domain.item()
        epoch_acc_domain += acc_domain.item()
        epoch_f1_domain += f1_domain.item()

    print(f"Epoch {epoch+1} Classifier: Loss: {epoch_loss_classifier/len(train_loader):.3f} | Accuracy: {epoch_acc_classifier/len(train_loader):.3f} | F1-Score: {epoch_f1_classifier/len(train_loader):.3f}")
    print(f"Epoch {epoch+1} Domain Discriminator: Loss: {epoch_loss_domain/len(train_loader):.3f} | Accuracy: {epoch_acc_domain/len(train_loader):.3f} | F1-Score: {epoch_f1_domain/len(train_loader):.3f}")


Epoch 1 Training: 100%|██████████| 430/430 [01:19<00:00,  5.41it/s]


Epoch 1 Classifier: Loss: 1.476 | Accuracy: 0.430 | F1-Score: 0.538
Epoch 1 Domain Discriminator: Loss: 0.692 | Accuracy: 0.549 | F1-Score: 0.205


Epoch 2 Training: 100%|██████████| 430/430 [01:18<00:00,  5.50it/s]


Epoch 2 Classifier: Loss: 1.292 | Accuracy: 0.527 | F1-Score: 0.582
Epoch 2 Domain Discriminator: Loss: 0.677 | Accuracy: 0.562 | F1-Score: 0.260


Epoch 3 Training: 100%|██████████| 430/430 [01:20<00:00,  5.35it/s]


...

Epoch 21 Classifier: Loss: 0.817 | Accuracy: 0.810 | F1-Score: 0.778
Epoch 21 Domain Discriminator: Loss: 0.697 | Accuracy: 0.551 | F1-Score: 0.329


Epoch 22 Training: 100%|██████████| 430/430 [01:22<00:00,  5.22it/s]


Epoch 22 Classifier: Loss: 0.767 | Accuracy: 0.819 | F1-Score: 0.787
Epoch 22 Domain Discriminator: Loss: 0.683 | Accuracy: 0.562 | F1-Score: 0.222


Epoch 23 Training: 100%|██████████| 430/430 [01:19<00:00,  5.39it/s]

Epoch 23 Classifier: Loss: 0.736 | Accuracy: 0.829 | F1-Score: 0.796
Epoch 23 Domain Discriminator: Loss: 0.679 | Accuracy: 0.564 | F1-Score: 0.203


# Kaggle =0.83

In [ ]:
from tqdm import tqdm
num_epochs = 20
alpha = 5  # Gradient reversal scale. Adjust based on your needs.


clip_value = 1

for epoch in range(num_epochs):
    epoch_loss_classifier = 0
    epoch_acc_classifier = 0
    epoch_f1_classifier = 0
    epoch_loss_domain = 0
    epoch_acc_domain = 0
    epoch_f1_domain = 0

    model.train()

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} Training"):
        texts, labels, domains, text_lengths = batch
        texts = texts.to(device)
        labels = labels.to(device)
        domains = domains.to(device)
        text_lengths = torch.tensor(text_lengths).long()  # Keep it on CPU

        batch_weights = get_batch_weights(labels, class_weights_domain2).to(device)

        optimizer.zero_grad()

        # Get the classifier's predictions
        predictions = model(texts, text_lengths)
        batch_criterion = nn.BCEWithLogitsLoss(pos_weight=batch_weights.squeeze()).to(device)
        loss_classifier = batch_criterion(predictions, labels)
        acc_classifier = binary_accuracy(predictions, labels)
        f1_classifier = compute_f1(predictions, labels)

        # Get the domain discriminator's predictions
        domain_predictions = model(texts, text_lengths, apply_discriminator=True, alpha=alpha)
        loss_domain = domain_adaptation_criterion(domain_predictions, domains)
        acc_domain = binary_accuracy(domain_predictions, domains)
        f1_domain = compute_f1(domain_predictions, domains)

        combined_loss = loss_classifier + loss_domain  # Removed reverse_gradient as we should apply gradient reversal on features and not the loss
        combined_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()

        epoch_loss_classifier += loss_classifier.item()
        epoch_acc_classifier += acc_classifier.item()
        epoch_f1_classifier += f1_classifier.item()
        epoch_loss_domain += loss_domain.item()
        epoch_acc_domain += acc_domain.item()
        epoch_f1_domain += f1_domain.item()

    print(f"Epoch {epoch+1} Classifier: Loss: {epoch_loss_classifier/len(train_loader):.3f} | Accuracy: {epoch_acc_classifier/len(train_loader):.3f} | F1-Score: {epoch_f1_classifier/len(train_loader):.3f}")
    print(f"Epoch {epoch+1} Domain Discriminator: Loss: {epoch_loss_domain/len(train_loader):.3f} | Accuracy: {epoch_acc_domain/len(train_loader):.3f} | F1-Score: {epoch_f1_domain/len(train_loader):.3f}")


Epoch 1 Training: 100%|██████████| 477/477 [01:25<00:00,  5.58it/s]


Epoch 1 Classifier: Loss: 1.435 | Accuracy: 0.465 | F1-Score: 0.534
Epoch 1 Domain Discriminator: Loss: 0.708 | Accuracy: 0.525 | F1-Score: 0.443


Epoch 2 Training: 100%|██████████| 477/477 [01:24<00:00,  5.67it/s]


Epoch 2 Classifier: Loss: 1.243 | Accuracy: 0.588 | F1-Score: 0.601
Epoch 2 Domain Discriminator: Loss: 0.681 | Accuracy: 0.550 | F1-Score: 0.502


Epoch 3 Training: 100%|██████████| 477/477 [01:25<00:00,  5.60it/s]


...

Epoch 18 Classifier: Loss: 0.806 | Accuracy: 0.805 | F1-Score: 0.760
Epoch 18 Domain Discriminator: Loss: 0.686 | Accuracy: 0.564 | F1-Score: 0.546


Epoch 19 Training: 100%|██████████| 477/477 [01:23<00:00,  5.72it/s]


Epoch 19 Classifier: Loss: 0.770 | Accuracy: 0.812 | F1-Score: 0.768
Epoch 19 Domain Discriminator: Loss: 0.684 | Accuracy: 0.572 | F1-Score: 0.567


Epoch 20 Training: 100%|██████████| 477/477 [01:24<00:00,  5.67it/s]

Epoch 20 Classifier: Loss: 0.750 | Accuracy: 0.824 | F1-Score: 0.780
Epoch 20 Domain Discriminator: Loss: 0.685 | Accuracy: 0.568 | F1-Score: 0.554


# above Kaggle=82

# Evaluation

In [14]:
def evaluate_model(model, valid_loader, device):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation for efficiency
        for batch in valid_loader:
            texts, labels, domains, text_lengths = batch
            texts = texts.to(device)
            labels = labels.float().to(device)

            # Compute model predictions
            predictions = model(texts, text_lengths)
            if predictions.dim() > 1 and predictions.size(1) == 1:
                predictions = predictions.squeeze(1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    binary_predictions = [1 if p >= 0.5 else 0 for p in all_predictions]

    # Convert lists to tensors
    tensor_predictions = torch.tensor(binary_predictions)
    tensor_labels = torch.tensor(all_labels)

    accuracy = binary_accuracy(tensor_predictions, tensor_labels)

    # Compute F1-Score
    f1 = f1_score(all_labels, binary_predictions, average='macro')

    return accuracy, f1

# After the training loop, evaluate on validation set
valid_accuracy, valid_f1 = evaluate_model(model, valid_loader, device)
print(f"Validation Accuracy: {valid_accuracy:.4f}")
print(f"Validation F1-Score: {valid_f1:.4f}")


Validation Accuracy: 0.8612
Validation F1-Score: 0.8519


# Send to Kaggle

In [15]:
import csv
import json

# Load the test data
with open('test_set.json', 'r') as f:
    test_data = [json.loads(line) for line in f]
# Evaluate on test data
model.eval()
test_results = []

In [16]:
with torch.no_grad():
    for entry in tqdm(test_data, desc="Evaluating Test Data"): # change from test_set to test_data
        text = entry["text"]
        text_tensor = torch.tensor(text).unsqueeze(0).to(device)  # Adding an extra batch dimension
        text_length = torch.tensor([len(text)])  # Sequence length for current entry

        # Pass the sequence and its length to the model
        prediction = model(text_tensor, text_length)
        if prediction.dim() > 1 and prediction.size(1) == 1:
            prediction = prediction.squeeze(1)
        prediction = torch.sigmoid(prediction).item()  # Convert raw score to value between 0 and 1

        # Classify the texts
        class_label = 1 if prediction >= 0.5 else 0

        test_results.append({
            "id": entry["id"],
            "class": class_label
        })

Evaluating Test Data: 100%|██████████| 1000/1000 [00:05<00:00, 198.87it/s]


In [17]:
# Write results to CSV
with open('results.csv', 'w', newline='') as csvfile:
    fieldnames = ['id', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in test_results:
        writer.writerow(result)